Import library

In [1]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from torchvision.transforms import v2
import torch.nn.functional as F

Custom Class for Load Data

In [ ]:
class CSVLabelDataset_Train(Dataset):
    def __init__(self, csv_dir, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_dir)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_name = self.annotations.iloc[idx]['file_name'] 
        label = int(self.annotations.iloc[idx]['label'])     
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            raise e   
        if self.transform:
            image = self.transform(image)
            
        return image, label

class CSVLabelDataset_Test(Dataset):
    def __init__(self, csv_dir, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_dir)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = self.annotations.iloc[idx]['file_name'] 
        id = str(self.annotations.iloc[idx]['id'])
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            raise e   
        if self.transform:
            image = self.transform(image)
        
        return image, id

Model train

In [ ]:
#set model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

BATCH_SIZE = 32
PATIENCE = 8
epochs = 20

model = models.swin_t(weights="DEFAULT")
model.head = torch.nn.Linear(model.head.in_features, 5)
model.to(device)

df = pd.read_csv('train.csv')

counts = df['label'].value_counts().sort_index()
class_weights = 1 / counts
class_weights = class_weights / class_weights.sum()
weights_tensor = torch.tensor(class_weights.values, dtype=torch.float32)

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.06, weight=weights_tensor.to(device))
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, weight_decay=0.02)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=PATIENCE)

scaler = torch.amp.GradScaler(device)

#set transform
imagenet_norm = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224),
        transforms.TrivialAugmentWide(),
        transforms.ToTensor(),
        imagenet_norm,
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        imagenet_norm,
    ]),
    'test': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        imagenet_norm,
    ])
}

cutmix = v2.CutMix(num_classes=5)
mixup = v2.MixUp(num_classes=5)

cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

#load data
csv_files = {
    'train': 'train.csv',
    'val': 'val.csv', 
    'test': 'test.csv'
}

img_dirs = {
    'train': 'train/',
    'val': 'val/',
    'test': 'test/'
}

datasets = {
    x: CSVLabelDataset_Train(
        csv_dir=csv_files[x],
        img_dir=img_dirs[x], 
        transform=data_transforms[x]
    ) if x in ['train', 'val'] else CSVLabelDataset_Test(
        csv_dir=csv_files[x],
        img_dir=img_dirs[x],
        transform=data_transforms[x]
    ) for x in ['train', 'val', 'test']
}

dataloaders = {
    x: DataLoader(
        datasets[x],
        batch_size=32,
        shuffle=(x == 'train'),  
    ) for x in ['train', 'val', 'test']
}

no_improve = 0
best_f1 = 0

#run epoch
for epoch in range(epochs):
    model.train()
    train_loss = 0

    for imgs, labels in tqdm(dataloaders['train']):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        imgs, soft_labels = cutmix_or_mixup(imgs, labels)

        with torch.amp.autocast(device):
            outputs = model(imgs)
            log_softmax_outputs = F.log_softmax(outputs, dim=1)
            loss = F.kl_div(log_softmax_outputs, soft_labels, reduction='batchmean')

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()

    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(dataloaders['val']):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = outputs.max(1)

            preds.extend(predicted.cpu().numpy())
            trues.extend(labels.cpu().numpy())

    f1 = f1_score(trues, preds, average="macro")
    scheduler.step(f1)

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss/len(dataloaders['train']):.4f} | "
          f"Val F1: {f1:.4f}")
    print("\nConfusion Matrix:")
    cm = confusion_matrix(trues, preds)
    print(cm)

    with np.errstate(divide='ignore', invalid='ignore'):
      cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
      cm_norm = np.nan_to_num(cm_norm)

    print("\nNormalized Confusion Matrix (% per class):")
    print(cm_norm)

    if f1 > best_f1:
        best_f1 = f1
        no_improve = 0
        torch.save(model.state_dict(), "model.pth")
        print("Improved F1")

    else:
        no_improve += 1
        print(f"No improvement ({no_improve}/{PATIENCE})")

    if no_improve >= PATIENCE:
        print("Early stopped")
        break

print("Best Validation F1:", best_f1)

Using device: cuda


/tmp/ipython-input-2911425189.py:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


  0%|          | 0/986 [00:00<?, ?it/s]

/tmp/ipython-input-2911425189.py:87: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


  0%|          | 0/775 [00:00<?, ?it/s]

...

Epoch 19/20 | Train Loss: 0.3860 | Val F1: 0.7833

Confusion Matrix:
[[3401    6   15    7    7]
 [ 188 3559  127  174  146]
 [ 360   34 5350  802  135]
 [ 125   48  526 2129   84]
 [ 557  673  575  595 5149]]

Normalized Confusion Matrix (% per class):
[[0.98981374 0.00174622 0.00436554 0.00203725 0.00203725]
 [0.04482594 0.84859323 0.03028135 0.04148784 0.03481164]
 [0.05388415 0.00508906 0.80077833 0.12004191 0.02020656]
 [0.04292582 0.01648352 0.18063187 0.73111264 0.02884615]
 [0.07378461 0.08915088 0.07616903 0.07881839 0.6820771 ]]
No improvement (7/8)


  0%|          | 0/986 [00:00<?, ?it/s]

/tmp/ipython-input-2911425189.py:87: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


  0%|          | 0/775 [00:00<?, ?it/s]

Epoch 20/20 | Train Loss: 0.3804 | Val F1: 0.7939

Confusion Matrix:
[[3380    6   20    8   22]
 [ 104 3595   96  235  164]
 [ 214   53 5069 1190  155]
 [  73   57  416 2246  120]
 [ 486  436  478  651 5498]]

Normalized Confusion Matrix (% per class):
[[0.98370198 0.00174622 0.00582072 0.00232829 0.00640279]
 [0.02479733 0.85717692 0.02288984 0.05603243 0.03910348]
 [0.03203113 0.00793294 0.75871875 0.17811705 0.02320012]
 [0.02506868 0.01957418 0.14285714 0.77129121 0.04120879]
 [0.06437939 0.05775599 0.06331964 0.08623659 0.72830839]]
Improved F1
Best Validation F1: 0.7939097491023525


Evaluation predict

In [ ]:
#eva
model.load_state_dict(torch.load("model.pth"))
model.eval()

#predict
ans_ids =[]
y_preds = []
with torch.no_grad():
    for img, id in tqdm(dataloaders['test']):
        outputs = model(img)
        _, predicted = outputs.max(1)
        ans_ids.extend(id)
        y_preds.extend(predicted.cpu().numpy())

#save answer
df_task4_ans = pd.DataFrame({"id": ans_ids, "label": y_preds})
df_task4_ans.to_csv("task4_ans.csv", index=False)

df_ans = pd.read_csv("Answer.csv")
df_ans['task4'] = df_task4_ans['label']
df_ans.to_csv("Answer.csv", index=False)

  0%|          | 0/810 [00:00<?, ?it/s]